In [2]:
import numpy as np
import pandas as pd
from scrapy.selector import Selector
from selenium import webdriver 
from selenium.webdriver.common.by import By
#from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Instantiate the data required in lists
users_list = ['ur117926588', 'ur3608743']
user_id = []
movie_title_list = []
year_list = []
review_title_list = []
review_list = []
error_msg = []

In [4]:
for user in range(len(users_list)):
    print("Scraping for User {}".format(user+1))
    driver = webdriver.Chrome('chromedriver.exe')
    url = 'https://www.imdb.com/user/{}/reviews'.format(users_list[user])
    time.sleep(1)
    driver.get(url)
    time.sleep(1)
    
    sel = Selector(text = driver.page_source)
    num_of_reviews = sel.css(".header span::text").extract_first().replace(',','').split(' ')[0]
    more_review_pages = int(int(num_of_reviews)/25)
    
    user_id += [users_list[user] for i in range(int(num_of_reviews))]
    
    # Loading all the reviews in the single page before scraping
    for i in tqdm(range(more_review_pages)):
        try:
            css_selector = 'load-more-trigger'
            driver.find_element(By.ID, css_selector).click()
            time.sleep(2)
        except:
            pass
    
    reviews = driver.find_elements(By.CSS_SELECTOR, 'div.review-container')
    for review in tqdm(reviews):
        try:
            sel2 = Selector(text = review.get_attribute('innerHTML'))
            try:
                movie_title = sel2.css('.lister-item-header a::text').extract_first().strip()
            except:
                movie_title = np.NaN
            try:
                year = sel2.css('.lister-item-year.text-muted.unbold::text').extract_first().strip().replace('(','').replace(')','')
            except:
                year = np.NaN
            try:
                review_title = sel2.css('.title::text').extract_first().strip()
            except:
                review_title = np.NaN
            try:
                review = sel2.css('.text.show-more__control::text').extract_first()
            except:
                review = np.NaN
                
            movie_title_list.append(movie_title)
            year_list.append(year)
            review_title_list.append(review_title)
            review_list.append(review)
        except Exception as e:
            error_msg.append(e)

Scraping for User 1


100%|██████████| 1242/1242 [00:21<00:00, 58.17it/s]


Scraping for User 2


100%|██████████| 888/888 [00:11<00:00, 75.43it/s] 


In [5]:
# Storing all data in dataframe
reviews_df = pd.DataFrame({
    "UserID": user_id,
    "Movie_Title": movie_title_list,
    "Year": year_list,
    "Review_Title": review_title_list,
    "Review": review_list})

In [6]:
# Export dataset
reviews_df.to_csv(path_or_buf = "users_reviews.csv"
                          , index = False )